- Documentation
    - [Getting Started Developing with Python and DynamoDB](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.html)
    - [Query Data Examples](https://www.fernandomc.com/posts/ten-examples-of-getting-data-from-dynamodb-with-python-and-boto3/);  [CheatSheet](https://dynobase.dev/dynamodb-python-with-boto3/)
    

- [Setting Up DynamoDB Local](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/DynamoDBLocal.html)

```
$ cat start_dynamodb 
java -Djava.library.path=~/projects/aws-cert/dynamodb/DynamoDBLocal_lib -jar ~/projects/aws-cert/dynamodb/DynamoDBLocal.jar -sharedDb

export DYNAMO_ENDPOINT="http://localhost:8000"

```

- [A GUI for Local DynamoDB— dynamodb-admin](https://medium.com/swlh/a-gui-for-local-dynamodb-dynamodb-admin-b16998323f8e)

```
$ sudo npm install -g npm
$ dynamodb-admin

dynamodb-admin listening on http://localhost:8001 
```




In [39]:
from __future__ import print_function # Python 2/3 compatibility
import json
import decimal

import boto3
from boto3.dynamodb.conditions import Key,Attr
from botocore.exceptions import ClientError

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if abs(o) % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)
    

### Step 0: List Tables


In [ ]:
DYNAMO_ENDPOINT = "http://localhost:8000"

# Get the service resource.
dynamodb = boto3.resource('dynamodb', endpoint_url=DYNAMO_ENDPOINT)

In [49]:
table_names = [table.name for table in dynamodb.tables.all()]
table_names

['Movies', 'UserImages']

### [Step 1: Create a Table](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.01.html)



In [2]:
TABLE_NAME = 'Movies'

In [3]:
def create_movie_table(table_name, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url=DYNAMO_ENDPOINT)
        
    try:
        table = dynamodb.Table(table_name)
        status = table.table_status
    except:

        # Error thrown when creating a pre-existing table
        # ResourceInUseException: An error occurred (ResourceInUseException) when calling the CreateTable operation: 
        # Cannot create preexisting table
        table = dynamodb.create_table(
            TableName=table_name,
            KeySchema=[
                {
                    'AttributeName': 'year',
                    'KeyType': 'HASH'  # Partition key
                },
                {
                    'AttributeName': 'title',
                    'KeyType': 'RANGE'  # Sort key
                }
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'year',
                    'AttributeType': 'N'
                },
                {
                    'AttributeName': 'title',
                    'AttributeType': 'S'
                },

            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        )
    return table

In [4]:
def load_movies(movies, table_name='Movies', dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table(table_name)
    with table.batch_writer() as batch:
        for movie in movies:
            year = int(movie['year'])
            title = movie['title']
            print("Adding movie:", year, title)
            batch.put_item(Item=movie)
        

In [5]:
table = create_movie_table(table_name=TABLE_NAME, dynamodb=dynamodb)
print("Table status:", table.table_status)

Table status: ACTIVE


### [Step 2: Load Sample Data](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.02.html)

In [6]:
with open("moviedata.json") as json_file:
    movie_list = json.load(json_file, parse_float=decimal.Decimal)
load_movies(movie_list, table_name=TABLE_NAME)

Adding movie: 2013 Rush
Adding movie: 2013 Prisoners
Adding movie: 2013 The Hunger Games: Catching Fire
Adding movie: 2013 Thor: The Dark World
Adding movie: 2013 This Is the End
Adding movie: 2013 Insidious: Chapter 2
Adding movie: 2013 World War Z
Adding movie: 2014 X-Men: Days of Future Past
Adding movie: 2014 Transformers: Age of Extinction
Adding movie: 2013 Now You See Me
Adding movie: 2013 Gravity
Adding movie: 2013 We're the Millers
Adding movie: 2013 Riddick
Adding movie: 2013 The Family
Adding movie: 2013 Star Trek Into Darkness
Adding movie: 2013 After Earth
Adding movie: 2013 The Great Gatsby
Adding movie: 2014 Divergent
Adding movie: 2013 We Are What We Are
Adding movie: 2013 Iron Man 3
Adding movie: 2014 The Amazing Spider-Man 2
Adding movie: 2013 Curse of Chucky
Adding movie: 2013 The Conjuring
Adding movie: 2013 Oldboy
Adding movie: 2013 Escape Plan
Adding movie: 2013 Elysium
Adding movie: 2013 Cloudy with a Chance of Meatballs 2
Adding movie: 2014 RoboCop
Adding movie:

Adding movie: 2001 Zoolander
Adding movie: 2003 Finding Nemo
Adding movie: 2009 (500) Days of Summer
Adding movie: 1966 Il buono, il brutto, il cattivo.
Adding movie: 2012 Life of Pi
Adding movie: 2013 The Big Wedding
Adding movie: 1999 The Matrix
Adding movie: 2014 Vampire Academy
Adding movie: 2010 Iron Man 2
Adding movie: 1994 Forrest Gump
Adding movie: 2013 Parker
Adding movie: 2004 The Chronicles of Riddick
Adding movie: 1994 The Lion King
Adding movie: 2012 Jack Reacher
Adding movie: 2011 The Girl with the Dragon Tattoo
Adding movie: 2009 Avatar
Adding movie: 2012 End of Watch
Adding movie: 2012 Snow White and the Huntsman
Adding movie: 2013 Mama
Adding movie: 2012 This Is 40
Adding movie: 2007 Superbad
Adding movie: 2012 Much Ado About Nothing
Adding movie: 2013 Last Vegas
Adding movie: 2010 Black Swan
Adding movie: 2011 Drive Angry
Adding movie: 2007 Ratatouille
Adding movie: 2010 Grown Ups
Adding movie: 2000 Gladiator
Adding movie: 2012 Hotel Transylvania
Adding movie: 2009 Th

Adding movie: 2011 Hall Pass
Adding movie: 2010 Harry Potter and the Deathly Hallows: Part 1
Adding movie: 2009 The Last House on the Left
Adding movie: 2011 We Need to Talk About Kevin
Adding movie: 2013 Saving Mr. Banks
Adding movie: 2012 Ice Age: Continental Drift
Adding movie: 2009 He's Just Not That Into You
Adding movie: 1998 Fear and Loathing in Las Vegas
Adding movie: 2011 The Hangover Part II
Adding movie: 1963 The Great Escape
Adding movie: 2011 Colombiana
Adding movie: 2012 House at the End of the Street
Adding movie: 2006 Children of Men
Adding movie: 2002 My Big Fat Greek Wedding
Adding movie: 1994 The Little Rascals
Adding movie: 2014 22 Jump Street
Adding movie: 2012 Step Up Revolution
Adding movie: 2011 Tinker Tailor Soldier Spy
Adding movie: 2012 The Lucky One
Adding movie: 2011 The Three Musketeers
Adding movie: 2010 The Last Song
Adding movie: 1986 Aliens
Adding movie: 2011 Source Code
Adding movie: 2007 Stardust
Adding movie: 2003 Dumb and Dumberer: When Harry Met L

Adding movie: 2007 Ocean's Thirteen
Adding movie: 2007 Ghost Rider
Adding movie: 2012 Great Expectations
Adding movie: 2005 Walk the Line
Adding movie: 2012 American Mary
Adding movie: 1998 The Parent Trap
Adding movie: 2013 The Stranger Within
Adding movie: 2008 Jumper
Adding movie: 2011 Priest
Adding movie: 2001 Blow
Adding movie: 2006 Invincible
Adding movie: 2009 Transformers: Revenge of the Fallen
Adding movie: 2012 Lockout
Adding movie: 2011 The Twilight Saga: Breaking Dawn - Part 1
Adding movie: 2013 Delivery Man
Adding movie: 2012 The Five-Year Engagement
Adding movie: 1965 The Sound of Music
Adding movie: 1961 Breakfast at Tiffany's
Adding movie: 2005 Mr. & Mrs. Smith
Adding movie: 2007 Beowulf
Adding movie: 2008 RocknRolla
Adding movie: 2013 Welcome to the Punch
Adding movie: 2009 Friday the 13th
Adding movie: 2012 Stolen
Adding movie: 2009 Orphan
Adding movie: 2012 The Words
Adding movie: 2008 The Bank Job
Adding movie: 2003 Underworld
Adding movie: 2009 Knowing
Adding movie

Adding movie: 1975 Monty Python and the Holy Grail
Adding movie: 2005 Match Point
Adding movie: 2008 The Other Boleyn Girl
Adding movie: 2002 Van Wilder
Adding movie: 1997 Mononoke-hime
Adding movie: 2011 Cars 2
Adding movie: 2012 So Undercover
Adding movie: 2003 2 Fast 2 Furious
Adding movie: 2010 The Kids Are All Right
Adding movie: 2003 The Dreamers
Adding movie: 2011 Jane Eyre
Adding movie: 2011 Apollo 18
Adding movie: 1999 Girl, Interrupted
Adding movie: 2011 Haywire
Adding movie: 1963 The Birds
Adding movie: 1997 Austin Powers: International Man of Mystery
Adding movie: 2012 Playing for Keeps
Adding movie: 1987 Empire of the Sun
Adding movie: 2011 La piel que habito
Adding movie: 2011 Red Riding Hood
Adding movie: 2013 As I Lay Dying
Adding movie: 2012 Jewtopia
Adding movie: 2013 Short Term 12
Adding movie: 2012 ParaNorman
Adding movie: 2013 The Zero Theorem
Adding movie: 1989 Uncle Buck
Adding movie: 2009 The Human Centipede (First Sequence)
Adding movie: 2007 30 Days of Night
A

...

Adding movie: 1977 The Rescuers
Adding movie: 1989 Look Who's Talking
Adding movie: 1969 The Italian Job
Adding movie: 2004 Dead Man's Shoes
Adding movie: 1987 Summer School
Adding movie: 1997 Cop Land
Adding movie: 1931 Dracula
Adding movie: 1969 True Grit
Adding movie: 2012 2 Days in New York
Adding movie: 1988 Above the Law
Adding movie: 2001 The 51st State
Adding movie: 2012 Rebelle
Adding movie: 2001 The Pledge
Adding movie: 2012 Chasing Ice
Adding movie: 2008 The Oxford Murders
Adding movie: 2009 The Messenger
Adding movie: 2010 Tropa de Elite 2 - O Inimigo Agora E Outro
Adding movie: 2013 No se si cortarme las venas o dejarmelas largas
Adding movie: 2011 The Tunnel
Adding movie: 1999 Life
Adding movie: 2007 The Reaping
Adding movie: 1998 Mercury Rising
Adding movie: 2009 I Hope They Serve Beer in Hell
Adding movie: 1987 Superman IV: The Quest for Peace
Adding movie: 2008 What Just Happened
Adding movie: 1991 The Commitments
Adding movie: 1985 Friday the 13th: A New Beginning
Add

Adding movie: 1991 Naked Lunch
Adding movie: 2000 Before Night Falls
Adding movie: 2010 The Perfect Host
Adding movie: 1988 I'm Gonna Git You Sucka
Adding movie: 2012 The Samaritan
Adding movie: 1996 Jingle All the Way
Adding movie: 2013 Crazy Kind of Love
Adding movie: 1998 Deep Rising
Adding movie: 2005 The Weather Man
Adding movie: 1995 La cite des enfants perdus
Adding movie: 2006 Facing the Giants
Adding movie: 2000 The Skulls
Adding movie: 2012 Asterix et Obelix: Au service de Sa Majeste
Adding movie: 2010 Lottery Ticket
Adding movie: 1997 An American Werewolf in Paris
Adding movie: 1955 The Night of the Hunter
Adding movie: 1994 In the Mouth of Madness
Adding movie: 2011 Wrong Turn 4: Bloody Beginnings
Adding movie: 2002 Dirty Pretty Things
Adding movie: 2006 Huo Yuan Jia
Adding movie: 2013 Jimmy P.
Adding movie: 1968 Where Eagles Dare
Adding movie: 1999 Superstar
Adding movie: 2008 Cadillac Records
Adding movie: 2014 Jane Got a Gun
Adding movie: 1995 First Knight
Adding movie: 

Adding movie: 2005 Grizzly Man
Adding movie: 2004 Bride & Prejudice
Adding movie: 2013 Copperhead
Adding movie: 1984 The Natural
Adding movie: 1999 Arlington Road
Adding movie: 1978 Every Which Way But Loose
Adding movie: 1997 Mimic
Adding movie: 1999 Bringing Out the Dead
Adding movie: 2011 Damsels in Distress
Adding movie: 1981 History of the World: Part I
Adding movie: 2012 Sleepwalk with Me
Adding movie: 2009 Doghouse
Adding movie: 1983 Twilight Zone: The Movie
Adding movie: 1998 Celebrity
Adding movie: 1992 School Ties
Adding movie: 2011 Hoodwinked Too! Hood vs. Evil
Adding movie: 1994 Exotica
Adding movie: 2006 The Butterfly Effect 2
Adding movie: 2011 Absentia
Adding movie: 2011 The Resident
Adding movie: 1978 Jaws 2
Adding movie: 1999 The Astronaut's Wife
Adding movie: 2003 11:14
Adding movie: 1984 Streets of Fire
Adding movie: 2008 Frozen River
Adding movie: 1983 Lone Wolf McQuade
Adding movie: 2012 Fast Girls
Adding movie: 2013 Intersections
Adding movie: 1958 Cat on a Hot Ti

Adding movie: 2008 The Lazarus Project
Adding movie: 2001 The Shipping News
Adding movie: 2012 Noobz
Adding movie: 1950 Harvey
Adding movie: 2014 Untitled Woody Allen Project
Adding movie: 1996 Spy Hard
Adding movie: 1994 Andre
Adding movie: 1999 200 Cigarettes
Adding movie: 2007 Battle in Seattle
Adding movie: 1994 The Client
Adding movie: 2013 Summer in February
Adding movie: 2009 Bandslam
Adding movie: 1996 Set It Off
Adding movie: 1996 Evita
Adding movie: 2002 The Country Bears
Adding movie: 2010 Every Day
Adding movie: 1950 Tea for Two
Adding movie: 2013 Madras Cafe
Adding movie: 2012 Tower Block
Adding movie: 2006 Away from Her
Adding movie: 1994 New Nightmare
Adding movie: 2012 Therese Desqueyroux
Adding movie: 2014 Solace
Adding movie: 2000 Chopper
Adding movie: 1992 Boomerang
Adding movie: 1965 Those Magnificent Men in Their Flying Machines or How I Flew from London to Paris in 25 hours 11 minutes
Adding movie: 2009 St Trinian's 2: The Legend of Fritton's Gold
Adding movie: 19

Adding movie: 2006 Kidulthood
Adding movie: 2002 In America
Adding movie: 2018 Halloween III
Adding movie: 1984 Repo Man
Adding movie: 2013 McCanick
Adding movie: 2009 Dread
Adding movie: 2012 Grassroots
Adding movie: 2009 The Maiden Heist
Adding movie: 2012 Dead Season
Adding movie: 1986 The Transformers: The Movie
Adding movie: 1980 Coal Miner's Daughter
Adding movie: 1999 Brokedown Palace
Adding movie: 2012 Free Samples
Adding movie: 2013 Home Sweet Home
Adding movie: 2001 Say It Isn't So
Adding movie: 1998 Jack Frost
Adding movie: 2011 Floquet de Neu
Adding movie: 1995 Nick of Time
Adding movie: 1988 Vampire's Kiss
Adding movie: 1980 Private Benjamin
Adding movie: 2005 Manderlay
Adding movie: 2010 Ca$h
Adding movie: 1957 Man on Fire
Adding movie: 1999 Blue Streak
Adding movie: 2011 Guns, Girls and Gambling
Adding movie: 2012 Good Deeds
Adding movie: 1988 The Blob
Adding movie: 1921 The Kid
Adding movie: 1981 Arthur
Adding movie: 2013 Self Storage
Adding movie: 2012 Love Sick Love
A

In [7]:
len(movie_list), movie_list[:2]

(4609,
 [{'year': 2013,
   'title': 'Rush',
   'info': {'directors': ['Ron Howard'],
    'release_date': '2013-09-02T00:00:00Z',
    'rating': Decimal('8.3'),
    'genres': ['Action', 'Biography', 'Drama', 'Sport'],
    'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTQyMDE0MTY0OV5BMl5BanBnXkFtZTcwMjI2OTI0OQ@@._V1_SX400_.jpg',
    'plot': 'A re-creation of the merciless 1970s rivalry between Formula One rivals James Hunt and Niki Lauda.',
    'rank': 2,
    'running_time_secs': 7380,
    'actors': ['Daniel Bruhl', 'Chris Hemsworth', 'Olivia Wilde']}},
  {'year': 2013,
   'title': 'Prisoners',
   'info': {'directors': ['Denis Villeneuve'],
    'release_date': '2013-08-30T00:00:00Z',
    'rating': Decimal('8.2'),
    'genres': ['Crime', 'Drama', 'Thriller'],
    'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTg0NTIzMjQ1NV5BMl5BanBnXkFtZTcwNDc3MzM5OQ@@._V1_SX400_.jpg',
    'plot': "When Keller Dover's daughter and her friend go missing, he takes matters into his own hands as th

### [Step 3: CRUD - Create, Read, Update, and Delete an Item](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.03.html)

given (pk,sk), one can create/update/delete/get item for that primary_key (pk,sk)

#### Step 3.1: Create a New Item  - put_item()

In [8]:
# dynamodb = boto3.resource('dynamodb', region_name='us-west-2', endpoint_url="http://localhost:8000")

table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

response = table.put_item(
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':"Nothing happens at all.",
            'rating': decimal.Decimal(0)
        }
    }
)

print("put_item succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

put_item succeeded:
{
    "ResponseMetadata": {
        "RequestId": "25628f74-f6b0-475b-a81d-0a867ac12832",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "x-amz-crc32": "2745614147",
            "x-amzn-requestid": "25628f74-f6b0-475b-a81d-0a867ac12832",
            "content-length": "2",
            "server": "Jetty(8.1.12.v20130726)"
        },
        "RetryAttempts": 0
    }
}


creating the same New Item will not cause duplicate

In [9]:


title = "The Big New Movie"
year = 2015

response = table.put_item(
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':"Nothing happens at all.",
            'rating': decimal.Decimal(-10)
        }
    }
)

print("put_item succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))


put_item succeeded:
{
    "ResponseMetadata": {
        "RequestId": "b89cdc10-b8d8-4970-86d8-3fb611a90cc1",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "x-amz-crc32": "2745614147",
            "x-amzn-requestid": "b89cdc10-b8d8-4970-86d8-3fb611a90cc1",
            "content-length": "2",
            "server": "Jetty(8.1.12.v20130726)"
        },
        "RetryAttempts": 0
    }
}


#### Step 3.2: Read an Item - get_item()

In [10]:
table = dynamodb.Table('Movies')

In [11]:
table

dynamodb.Table(name='Movies')

In [12]:
title = "The Big New Movie"
year = 2015

try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    print("get_item succeeded:")
    print(json.dumps(item, indent=4, cls=DecimalEncoder))

get_item succeeded:
{
    "title": "The Big New Movie",
    "year": 2015,
    "info": {
        "rating": -10,
        "plot": "Nothing happens at all."
    }
}


#### Step 3.3: Update an Item

`put_item()`  does `update_item()`

In [13]:
table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015
rating = 5.5
plot = "Everything happens all at once."
actors = ["Larry", "Moe", "Curly"]

try:
    response = table.update_item(
        Key={
            'year': year,
            'title': title
        },
        UpdateExpression="set info.rating=:r, info.plot=:p, info.actors=:a",
        ExpressionAttributeValues={
            ':r': decimal.Decimal(rating),
            ':p': plot,
            ':a': actors
        },
        ReturnValues="UPDATED_NEW"
    )
except ClientError as e:
    print(e.response['Error']['Message'])

In [14]:
response

{'Attributes': {'info': {'rating': Decimal('5.5'),
   'actors': ['Larry', 'Moe', 'Curly'],
   'plot': 'Everything happens all at once.'}},
 'ResponseMetadata': {'RequestId': '258a044c-9ab5-4935-b764-f900d2afc531',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2682384945',
   'x-amzn-requestid': '258a044c-9ab5-4935-b764-f900d2afc531',
   'content-length': '156',
   'server': 'Jetty(8.1.12.v20130726)'},
  'RetryAttempts': 0}}

In [15]:
title = "The Big New Movie"
year = 2015

try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    
item

{'title': 'The Big New Movie',
 'year': Decimal('2015'),
 'info': {'rating': Decimal('5.5'),
  'actors': ['Larry', 'Moe', 'Curly'],
  'plot': 'Everything happens all at once.'}}

#####  Update an Item conditionally

In [16]:
actor_count = 2
try:
    response = table.update_item(
        Key={
            'year': year,
            'title': title
        },
        UpdateExpression="remove info.actors[0]",
        ConditionExpression="size(info.actors) > :num",
        ExpressionAttributeValues={':num': actor_count},
        ReturnValues="UPDATED_NEW"
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise

In [17]:
# update successful if actor_count < 3: "Larry" is removed
# The conditional request failed if actor_count >= 3

In [18]:
response

{'Attributes': {'info': {'rating': Decimal('5.5'),
   'actors': ['Moe', 'Curly'],
   'plot': 'Everything happens all at once.'}},
 'ResponseMetadata': {'RequestId': 'dc950c7f-fd49-4de1-adc9-f1cf30962ff9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '1320246852',
   'x-amzn-requestid': 'dc950c7f-fd49-4de1-adc9-f1cf30962ff9',
   'content-length': '142',
   'server': 'Jetty(8.1.12.v20130726)'},
  'RetryAttempts': 0}}

#### Step 3.6: Delete an Item

In [19]:
year = 2013
title = "Prisoners"

In [20]:
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    
item

{'title': 'Prisoners',
 'year': Decimal('2013'),
 'info': {'actors': ['Hugh Jackman', 'Jake Gyllenhaal', 'Viola Davis'],
  'release_date': '2013-08-30T00:00:00Z',
  'plot': "When Keller Dover's daughter and her friend go missing, he takes matters into his own hands as the police pursue multiple leads and the pressure mounts. But just how far will this desperate father go to protect his family?",
  'genres': ['Crime', 'Drama', 'Thriller'],
  'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTg0NTIzMjQ1NV5BMl5BanBnXkFtZTcwNDc3MzM5OQ@@._V1_SX400_.jpg',
  'directors': ['Denis Villeneuve'],
  'rating': Decimal('8.2'),
  'rank': Decimal('3'),
  'running_time_secs': Decimal('9180')}}

In [21]:
try:
    response = table.delete_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        item = response['Item']

In [22]:
response

{'ResponseMetadata': {'RequestId': '257270bd-367d-484a-b3dd-7d3108d45341',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': '257270bd-367d-484a-b3dd-7d3108d45341',
   'content-length': '2',
   'server': 'Jetty(8.1.12.v20130726)'},
  'RetryAttempts': 0}}

In [23]:
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    
item

KeyError: 'Item'

##### Delete an Item conditionally

In [24]:
year = 2013
title = "The Hunger Games: Catching Fire"
rank = 5

In [25]:
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    
item

{'title': 'The Hunger Games: Catching Fire',
 'year': Decimal('2013'),
 'info': {'actors': ['Jennifer Lawrence', 'Josh Hutcherson', 'Liam Hemsworth'],
  'release_date': '2013-11-11T00:00:00Z',
  'plot': 'Katniss Everdeen and Peeta Mellark become targets of the Capitol after their victory in the 74th Hunger Games sparks a rebellion in the Districts of Panem.',
  'genres': ['Action', 'Adventure', 'Sci-Fi', 'Thriller'],
  'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTAyMjQ3OTAxMzNeQTJeQWpwZ15BbWU4MDU0NzA1MzAx._V1_SX400_.jpg',
  'directors': ['Francis Lawrence'],
  'rank': Decimal('4'),
  'running_time_secs': Decimal('8760')}}

In [26]:
try:
    response = table.delete_item(
        Key={
            'year': year,
            'title': title
        },
        ConditionExpression="info.#rnk <= :val",
        ExpressionAttributeValues={
            ":val": decimal.Decimal(rank)
        },
        ExpressionAttributeNames={
            "#rnk": "rank"
        }
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise

In [27]:
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    
item

KeyError: 'Item'

### [Step 4: Query and Scan the Data](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.04.html)


- [Working with Queries](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Query.html)
    - begins_with
    - between
    - eq
    - gt 
    - gte
    - lt
    - lte
    - mro

#### Query - All Movies Released in a Year

* matching by partition key and sort key

In [28]:
year = 2009
table = dynamodb.Table('Movies')
response = table.query(
    # KeyConditionExpression=Key('year').eq(year) & Key('title').begins_with('H')
    # KeyConditionExpression=Key('year').eq(year) & Key('title').gt('H')
    KeyConditionExpression=Key('year').eq(year) & Key('title').between('B', 'D')
)

In [29]:
response

{'Items': [{'title': 'Bakjwi',
   'year': Decimal('2009'),
   'info': {'actors': ['Kang-ho Song', 'Ok-bin Kim', 'Hae-suk Kim'],
    'release_date': '2009-04-30T00:00:00Z',
    'plot': 'Through a failed medical experiment, a priest is stricken with vampirism and is forced to abandon his ascetic ways.',
    'genres': ['Drama', 'Horror', 'Thriller'],
    'image_url': 'http://ia.media-imdb.com/images/M/MV5BNDgwODY0MjM3OV5BMl5BanBnXkFtZTcwNzk3MTY2Mg@@._V1_SX400_.jpg',
    'directors': ['Chan-wook Park'],
    'rating': Decimal('7.1'),
    'rank': Decimal('3837'),
    'running_time_secs': Decimal('7980')}},
  {'title': 'Bandslam',
   'year': Decimal('2009'),
   'info': {'actors': ['Aly Michalka', 'Vanessa Hudgens', 'Gaelan Connell'],
    'release_date': '2009-08-06T00:00:00Z',
    'plot': 'A new kid in town assembles a fledgling rock band -- together, they achieve their dreams and compete against the best in the biggest event of the year, a battle of the bands.',
    'genres': ['Comedy', 'Dra

In [30]:
len(response['Items']), response['Items'][:2]

(27,
 [{'title': 'Bakjwi',
   'year': Decimal('2009'),
   'info': {'actors': ['Kang-ho Song', 'Ok-bin Kim', 'Hae-suk Kim'],
    'release_date': '2009-04-30T00:00:00Z',
    'plot': 'Through a failed medical experiment, a priest is stricken with vampirism and is forced to abandon his ascetic ways.',
    'genres': ['Drama', 'Horror', 'Thriller'],
    'image_url': 'http://ia.media-imdb.com/images/M/MV5BNDgwODY0MjM3OV5BMl5BanBnXkFtZTcwNzk3MTY2Mg@@._V1_SX400_.jpg',
    'directors': ['Chan-wook Park'],
    'rating': Decimal('7.1'),
    'rank': Decimal('3837'),
    'running_time_secs': Decimal('7980')}},
  {'title': 'Bandslam',
   'year': Decimal('2009'),
   'info': {'actors': ['Aly Michalka', 'Vanessa Hudgens', 'Gaelan Connell'],
    'release_date': '2009-08-06T00:00:00Z',
    'plot': 'A new kid in town assembles a fledgling rock band -- together, they achieve their dreams and compete against the best in the biggest event of the year, a battle of the bands.',
    'genres': ['Comedy', 'Drama',

#### Query with pagination

* `ScanIndexForward` controls sort order
* `LastEvaluatedKey` in `response` indicates there are more results to fetch

In [43]:
year = 1992
title_range = ('B', 'E')

# Expression attribute names can only reference items in the projection expression.
# year is a reserved word, use ExpressionAttributeNames to remap it
query_kwargs = {
    "ProjectionExpression": "#yr, title, info.genres, info.actors[0]",
    "ExpressionAttributeNames": {"#yr": "year"},
    "KeyConditionExpression": Key('year').eq(year) & Key('title').between(*title_range),
    "ScanIndexForward": False  # sort: ascending if True, descending if False 
}

## paginate
done, start_key = False, None
items = []
while not done:
    if start_key:
        query_kwargs['ExclusiveStartKey'] = start_key
    response = table.query(**query_kwargs)
    items.extend(response.get('Items', []))
    start_key = response.get('LastEvaluatedKey', None)
    done = start_key is None
    
len(items), items

(12,
 [{'title': 'Death Becomes Her',
   'year': Decimal('1992'),
   'info': {'actors': ['Meryl Streep'], 'genres': ['Comedy', 'Fantasy']}},
  {'title': 'Damage',
   'year': Decimal('1992'),
   'info': {'actors': ['Jeremy Irons'], 'genres': ['Drama', 'Romance']}},
  {'title': 'Como agua para chocolate',
   'year': Decimal('1992'),
   'info': {'actors': ['Marco Leonardi'], 'genres': ['Drama', 'Romance']}},
  {'title': 'Chaplin',
   'year': Decimal('1992'),
   'info': {'actors': ['Robert Downey Jr.'],
    'genres': ['Biography', 'Drama']}},
  {'title': 'Captain Ron',
   'year': Decimal('1992'),
   'info': {'actors': ['Kurt Russell'], 'genres': ['Adventure', 'Comedy']}},
  {'title': 'Candyman',
   'year': Decimal('1992'),
   'info': {'actors': ['Virginia Madsen'], 'genres': ['Horror']}},
  {'title': 'Buffy the Vampire Slayer',
   'year': Decimal('1992'),
   'info': {'actors': ['Kristy Swanson'],
    'genres': ['Action', 'Comedy', 'Fantasy', 'Horror']}},
  {'title': 'Braindead',
   'year':

#### Scan with pagination

In [36]:
year_range = (1950, 1955)
query_kwargs = {
    'FilterExpression': Key('year').between(*year_range),
    'ProjectionExpression': "#yr, title, info.rating",
    'ExpressionAttributeNames': {"#yr": "year"}
}

items = []

done, start_key = False, None
while not done:
    if start_key:
        query_kwargs['ExclusiveStartKey'] = start_key
    response = table.scan(**query_kwargs)
    items.extend(response.get('Items', []))
    start_key = response.get('LastEvaluatedKey', None)
    done = start_key is None

len(items), items[:3]

(38,
 [{'title': 'High Noon',
   'year': Decimal('1952'),
   'info': {'rating': Decimal('8.2')}},
  {'title': "Singin' in the Rain",
   'year': Decimal('1952'),
   'info': {'rating': Decimal('8.4')}},
  {'title': 'The Member of the Wedding',
   'year': Decimal('1952'),
   'info': {'rating': Decimal('6.8')}}])

In [38]:
year_range = (1950, 1955)
query_kwargs = {
    'FilterExpression': Key('year').between(*year_range),
    'ProjectionExpression': "#yr, title, info.rating",
    'ExpressionAttributeNames': {"#yr": "year"}
}

items = []
response = table.scan(**query_kwargs)
items.extend(response['Items'])

# paginate in another way
while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'], **query_kwargs)
    items.extend(response['Items'])

len(items), items[:3]

(38,
 [{'title': 'High Noon',
   'year': Decimal('1952'),
   'info': {'rating': Decimal('8.2')}},
  {'title': "Singin' in the Rain",
   'year': Decimal('1952'),
   'info': {'rating': Decimal('8.4')}},
  {'title': 'The Member of the Wedding',
   'year': Decimal('1952'),
   'info': {'rating': Decimal('6.8')}}])

* FilterExpression - add condition on info.rating attribute

In [40]:
year_range = (1950, 1955)
query_kwargs = {
    'FilterExpression': Key('year').between(*year_range) & Attr('info.rating').gt(decimal.Decimal(8.0)),
    'ProjectionExpression': "#yr, title, info.rating",
    'ExpressionAttributeNames': {"#yr": "year"}
}

items = []
response = table.scan(**query_kwargs)
items.extend(response['Items'])

# paginate in another way
while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'], **query_kwargs)
    items.extend(response['Items'])

len(items), items[:3]

(13,
 [{'title': 'High Noon',
   'year': Decimal('1952'),
   'info': {'rating': Decimal('8.2')}},
  {'title': "Singin' in the Rain",
   'year': Decimal('1952'),
   'info': {'rating': Decimal('8.4')}},
  {'title': 'Dial M for Murder',
   'year': Decimal('1954'),
   'info': {'rating': Decimal('8.1')}}])

### Delete a Table


In [ ]:
table.delete()

### Local Secondary Index (LSI)

- https://highlandsolutions.com/blog/hands-on-examples-for-working-with-dynamodb-boto3-and-python


In [55]:
import boto3
from boto3.dynamodb.conditions import Key

def create_table_with_lsi():
    dynamodb = boto3.resource('dynamodb', endpoint_url=DYNAMO_ENDPOINT)
    
    table = dynamodb.create_table(
        TableName='Posts',
        KeySchema=[
            {
                'AttributeName': 'user_name',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'title',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'user_name',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'subject',
                'AttributeType': 'S'
            },
    
        ],
        LocalSecondaryIndexes=[
            {
                'IndexName': 'user_name_subject',
                'KeySchema': [
                    {
                        'AttributeName': 'user_name',
                        'KeyType': 'HASH'
                    },
                    {
                        'AttributeName': 'subject',
                        'KeyType': 'RANGE'
                    },
                ],
                'Projection': {
                    'ProjectionType': 'ALL'
                },
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 1,
            'WriteCapacityUnits': 1,
        }
    )

    print("Table status:", table.table_status)

def create_posts():
    post1 = {
        'title': "My favorite hiking spots",
        'user_name': 'jon_doe',
        'subject': 'hiking'
    }
    
    post2 = {
        'title': "My favorite recipes",
        'user_name': 'jon_doe',
        'subject': 'cooking'
    }
    
    post3 = {
        'title': "I love hiking!",
        'user_name': 'jane_doe',
        'subject': 'hiking'
    }
    
    dynamodb = boto3.resource('dynamodb', endpoint_url=DYNAMO_ENDPOINT)
    
    table = dynamodb.Table('Posts')
    
    table.put_item(Item=post1)
    table.put_item(Item=post2)
    table.put_item(Item=post3)

def query_data_with_lsi():
    dynamodb = boto3.resource('dynamodb', endpoint_url=DYNAMO_ENDPOINT)
    
    table = dynamodb.Table('Posts')
    
    response = table.query(
        IndexName='user_name_subject',
        KeyConditionExpression=
            Key('user_name').eq('jon_doe') & Key('subject').eq('hiking')
    )
    
    print(response['Items'][0])
    #{'subject': 'hiking', 'user_name': 'jon_doe', 'title': 'My favorite hiking spots'}

In [ ]:
create_table_with_lsi()
# wait for a while

In [54]:
create_posts()

In [56]:
query_data_with_lsi()

{'title': 'My favorite hiking spots', 'user_name': 'jon_doe', 'subject': 'hiking'}


### Global Secondary Index (GSI)

- https://highlandsolutions.com/blog/hands-on-examples-for-working-with-dynamodb-boto3-and-python


In [63]:
import boto3
from boto3.dynamodb.conditions import Key

def create_table_with_gsi():
    dynamodb = boto3.resource('dynamodb', endpoint_url=DYNAMO_ENDPOINT)
    
    table = dynamodb.create_table(
        TableName='Employees',
        KeySchema=[
            {
                'AttributeName': 'emp_id',
                'KeyType': 'HASH'
            },
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'emp_id',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'email',
                'AttributeType': 'S'
            },
    
        ],
        GlobalSecondaryIndexes=[
            {
                'IndexName': 'email',
                'KeySchema': [
                    {
                        'AttributeName': 'email',
                        'KeyType': 'HASH'
                    },
                ],
                'Projection': {
                    'ProjectionType': 'ALL'
                },
                'ProvisionedThroughput' :{
                    'ReadCapacityUnits': 1,
                    'WriteCapacityUnits': 1,
                }
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 1,
            'WriteCapacityUnits': 1,
        }
    )

    print("Table status:", table.table_status)
    
def create_employee():
    dynamodb = boto3.resource('dynamodb', endpoint_url=DYNAMO_ENDPOINT)
    
    table = dynamodb.Table('Employees')
    for i in range(20):
        user = {
            'emp_id': i,
            'first_name': 'Jon'+str(i),
            'last_name': 'Doe',
            'email': f'jdoe{str(i)}@test.com'
        }    
        table.put_item(Item=user)
    
def query_data_with_gsi():
    dynamodb = boto3.resource('dynamodb', endpoint_url=DYNAMO_ENDPOINT)
    
    table = dynamodb.Table('Employees')
    
    response = table.query(
        IndexName='email',
        KeyConditionExpression=Key('email').eq('jdoe7@test.com')
    )
    
    print(response['Items'])
    #{'last_name': 'Doe', 'email': 'jdoe@test.com', 'first_name': 'Jon', 'emp_id': Decimal('1')}


In [58]:
create_table_with_gsi()

Table status: ACTIVE


In [61]:
create_employee()

In [64]:
query_data_with_gsi()

[{'last_name': 'Doe', 'first_name': 'Jon7', 'email': 'jdoe7@test.com', 'emp_id': Decimal('7')}]
